# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [7]:
!pip install hvplot

In [8]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests


# Import API key
from api_keys import geoapify_key


In [9]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [27]:
%%capture --no-display

map = city_data_df.hvplot.points(
    "Lng", 
    "Lat",
    geo=True,
    size = "Humidity",
    tiles='ESRI'
).opts(width=700, height=500)

map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [35]:
# Narrow down cities that fit criteria and drop any results with null values
condition = city_data_df['Cloudiness']<40

ideal_weather = city_data_df[condition]

# Drop any rows with null values
ideal_weather_dropped = ideal_weather.dropna()
                             
# Display sample data
ideal_weather_dropped.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
6,6,albany,42.6001,-73.9662,10.36,74,29,2.10,US,1666108231
7,7,carnarvon,-24.8667,113.6333,20.02,76,0,7.46,AU,1666108232
9,9,saint-pierre,-21.3393,55.4781,22.82,68,0,5.14,RE,1666108236
12,12,trairi,-3.2778,-39.2689,31.07,51,34,8.09,BR,1666108038
20,20,avarua,-21.2078,-159.7750,22.03,73,20,2.57,CK,1666108220
22,22,guiratinga,-16.3494,-53.7583,38.35,14,22,3.10,BR,1666108243
29,29,saint-louis,16.3333,-15.0000,38.80,24,4,1.78,SN,1666108247


### Step 3: Create a new DataFrame called `hotel_df`.

In [37]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_dropped.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_name = None
hotel_df.insert((len(hotel_df.columns)-1), 'Hotel Name', hotel_name)

# Display sample data
hotel_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Hotel Name,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,None,1666108228
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,None,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,None,1666108230
6,6,albany,42.6001,-73.9662,10.36,74,29,2.10,US,None,1666108231
7,7,carnarvon,-24.8667,113.6333,20.02,76,0,7.46,AU,None,1666108232
...,...,...,...,...,...,...,...,...,...,...,...
564,564,redmond,47.6740,-122.1215,13.17,91,8,0.17,US,None,1666108556
565,565,wyndham,37.6985,-77.6125,12.81,52,3,1.79,US,None,1666108480
571,571,dargaville,-35.9333,173.8833,10.97,83,13,3.83,NZ,None,1666108563
575,575,iquique,-20.2208,-70.1431,17.02,68,6,3.50,CL,None,1666108566


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [40]:
# Set parameters to search for a hotel
radius = 10000
params = {
   "filter":None,
    "limit":1,
    "bias":None,
    "apiKey":geoapify_key,
    "categories": "accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index, 'Lat']
    longitude = hotel_df.loc[index, 'Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head(10)

Starting hotel search
faya - nearest hotel: No hotel found
farsund - nearest hotel: Rederiet Hotell
new norfolk - nearest hotel: Woodbridge on the Derwent
albany - nearest hotel: No hotel found
carnarvon - nearest hotel: No hotel found
saint-pierre - nearest hotel: Tropic Hotel
trairi - nearest hotel: Pousada Villa Aurora
avarua - nearest hotel: Paradise Inn
guiratinga - nearest hotel: No hotel found
saint-louis - nearest hotel: No hotel found
grimmen - nearest hotel: No hotel found
cape town - nearest hotel: Townhouse Hotel
stornoway - nearest hotel: County Hotel
margate - nearest hotel: The Reading Rooms
ancud - nearest hotel: Cabañas
collierville - nearest hotel: Days Inn & Suites by Wyndham Collierville Germantown Area
kapaa - nearest hotel: Pono Kai Resort
mogadishu - nearest hotel: Villa Rays Restaurant & Hotel
hobart - nearest hotel: Vibe Hotel Hobart
hilo - nearest hotel: Dolphin Bay Hotel
lasa - nearest hotel: Hotel "Drei Kreuz"
vaini - nearest hotel: Keleti International Reso

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Hotel Name,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,No hotel found,1666108228
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,Rederiet Hotell,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,Woodbridge on the Derwent,1666108230
6,6,albany,42.6001,-73.9662,10.36,74,29,2.10,US,No hotel found,1666108231
7,7,carnarvon,-24.8667,113.6333,20.02,76,0,7.46,AU,No hotel found,1666108232
9,9,saint-pierre,-21.3393,55.4781,22.82,68,0,5.14,RE,Tropic Hotel,1666108236
12,12,trairi,-3.2778,-39.2689,31.07,51,34,8.09,BR,Pousada Villa Aurora,1666108038
20,20,avarua,-21.2078,-159.7750,22.03,73,20,2.57,CK,Paradise Inn,1666108220
22,22,guiratinga,-16.3494,-53.7583,38.35,14,22,3.10,BR,No hotel found,1666108243
29,29,saint-louis,16.3333,-15.0000,38.80,24,4,1.78,SN,No hotel found,1666108247


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [45]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng", 
    "Lat",
    geo=True,
    size="Humidity",
    tiles='ESRI',
    hover_cols=['Country', 'Hotel Name']
).opts(width=700, height=500)


# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity,Country,Hotel Name)